In [40]:
import os 
import utils.visulaiser as visulaiser
from datasets import load_dataset, load_from_disk

from torch import nn
from tqdm import tqdm
import numpy as np
import torch
import copy
import matplotlib.pyplot as plt
from datasets import load_dataset
import re
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch.optim import AdamW
import torch.nn as nn
from torchvision.transforms import v2

from transformers import AutoModelForCausalLM, AutoTokenizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'

C:\Users\gsnik\AppData\Roaming\Python\Python311\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\gsnik\AppData\Roaming\Python\Python311\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any 

In [2]:
print(device)

cuda


# Model

In [3]:
model_name = "Qwen/Qwen2-Math-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
base_model = AutoModelForCausalLM.from_pretrained(model_name)

C:\Users\gsnik\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

C:\Users\gsnik\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gsnik\.cache\huggingface\hub\models--Qwen--Qwen2-Math-1.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

C:\Users\gsnik\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

In [4]:
# Save model and tokenizer to the current directory
base_model.save_pretrained("./Qwen2.5-0.5B-Instruct")
tokenizer.save_pretrained("./Qwen2.5-0.5B-Instruct")

('./Qwen2.5-0.5B-Instruct\\tokenizer_config.json',
 './Qwen2.5-0.5B-Instruct\\special_tokens_map.json',
 './Qwen2.5-0.5B-Instruct\\vocab.json',
 './Qwen2.5-0.5B-Instruct\\merges.txt',
 './Qwen2.5-0.5B-Instruct\\added_tokens.json',
 './Qwen2.5-0.5B-Instruct\\tokenizer.json')

In [66]:
class ModifiedModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.qwen = base_model.model
        self.regression = nn.Sequential(
            nn.Linear(1536, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, inputs):
        outputs = self.qwen(**inputs)
        cls_token = outputs.last_hidden_state[:, 0, :]
        return self.regression(cls_token)

# Data

In [45]:
# Load GSM8K dataset
# dataset = load_dataset("gsm8k", "main")
dataset = load_from_disk("./gsm8k_saved")

def extract_final_answer(answer):
    """
    Extracts only the numerical value after '####' in the answer field.
    """
    match = re.search(r"####\s*([\d\.]+)", answer)  # Match number after ####
    return float(match.group(1)) if match else 0  # Return extracted number

# Process training and test sets
for split in ["train", "test"]:
    dataset[split] = dataset[split].map(lambda example: {
        "question": example["question"],
        # "answer": tokenizer(extract_final_answer(example["answer"]),
        #                     padding='max_length',
        #                     truncation=True,
        #                     max_length=16,
        #                     return_tensors='pt').to(device),
        "answer": extract_final_answer(example["answer"])
    })

# Save processed dataset
dataset.save_to_disk("./gsm8k_cleaned")

# Print an example to verify
print(dataset["train"][0])

# Split into train and test sets
train_data = dataset["train"]
test_data = dataset["test"]

Saving the dataset (0/1 shards):   0%|          | 0/7473 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1319 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 72.0}


In [46]:
answers = []

for i in range(len(train_data)):
    answers.append(train_data[i]['answer'])

train_mean = np.mean(answers)
train_std = np.std(answers)

for i in range(len(train_data)):
    train_data[i]['answer'] = (train_data[i]['answer'] - train_mean) / train_std

for i in range(len(test_data)):
    test_data[i]['answer'] = (test_data[i]['answer'] - train_mean) / train_std

In [47]:
def format_example(example):
    # print(example)
    return f"Question: YOU ARE A EXPERT AT MATH. NOW ANSWER THIS QUESTION - {example['question']}. REPLY JUST THE FINAL ANSWER AS A NUMBER. Answer: "

# Tokenize data
def preprocess_function(examples):
    texts = format_example(examples)
    tokens = tokenizer(texts, 
                     padding="max_length", 
                     truncation=True, 
                     max_length=128, 
                     return_tensors="pt")
    return tokens

# Apply preprocessing
tokenized_train = train_data.map(preprocess_function, batched=False)
tokenized_test = test_data.map(preprocess_function, batched=False)

# Rename 
# tokenized_train = tokenized_train.remove_columns('question')
# tokenized_train = tokenized_train.rename_column('answer', 'labels')

# tokenized_test = tokenized_test.remove_columns('question')
# tokenized_test = tokenized_test.rename_column('answer', 'labels')

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [48]:
small_train_dataset = tokenized_train.shuffle(seed=42)#.select(range(1000))
small_eval_dataset = tokenized_test.shuffle(seed=42)

In [49]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=1)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=1)

# Explore Base Model

In [50]:
base_model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_

In [30]:
def print_model_predictions(model, tokenizer, dataset, device="cpu", num_samples=5):
    model.to(device)
    model.eval()
    dataset = load_from_disk(dataset)
    dataset = dataset['test']
    
    for i in range(min(num_samples, len(dataset))):
        example = dataset[56+i]
        input_text = "You are a math expert. Now answer this question - " + example["question"] + " Your answer should only contain the final answer as a number. Print final answer here: "
        target_output = example["answer"]
        
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        with torch.no_grad():
            output_ids = model.generate(**inputs, max_new_tokens=64)
        generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        print(f"Example {i+1}:\n")
        print(f"Input: {input_text}\n")
        print(f"Generated Answer: {generated_answer}\n")
        print(f"Target Output: {target_output}\n")
        print("-" * 50)
        if i == 20:
            break

In [31]:
print_model_predictions(base_model, tokenizer, "./gsm8k_saved", device, num_samples=2)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Example 1:

Input: You are a math expert. Now answer this question - Peter plans to go to the movies this week. He always gets a ticket for $7 and popcorn for $7. If he has 42 dollars for the week, how many times can he go to the movies? Your answer should only contain the final answer as a number. Print final answer here: 

Generated Answer: You are a math expert. Now answer this question - Peter plans to go to the movies this week. He always gets a ticket for $7 and popcorn for $7. If he has 42 dollars for the week, how many times can he go to the movies? Your answer should only contain the final answer as a number. Print final answer here: 3
To determine how many times Peter can go to the movies, we need to calculate the total cost for each visit and compare it to his weekly budget.

1. **Identify the cost per visit:**
   - Ticket: $7
   - Popcorn: $7
   - Total cost per visit

Target Output: It costs him $14 to go to the movies because 7 + 7 = <<7+7=14>>14
He can go to the movies 3

# PyTorch Bare Bones

In [70]:
def train(model, dataloader, loss_metric, optimizer, num_epochs):
    num_training_steps = num_epochs * len(dataloader)
    progress_bar = tqdm(range(num_training_steps))
    
    model = ModifiedModel(base_model)
    model = model.to(device)
    model.train()
    
    optimizer = AdamW(model.parameters(), lr=5e-3)
    loss_metric = nn.MSELoss()
    
    loss_arr = []
    
    for epoch in range(num_epochs):
        running_loss = 0
        for i, sample in enumerate(dataloader):
            # print(sample)
            batch = {k: torch.tensor(v).to(device) for k, v in sample.items() if k != 'answer' and k != 'question'}
            
            output = model(batch)
            if isinstance(output, tuple):  # Ensure proper indexing
                output = output[0]
            
            # Ensure shape consistency for loss calculation
            loss = loss_metric(output.view(-1, 1).float(), sample['answer'].view(-1, 1).to(device).float())
    
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            progress_bar.update(1)
    
            running_loss += loss.item()
            if (i % 100 == 0):
                print(output.item(), sample['answer'].item())
                print(f"Step {i}: Loss = {loss.item()}")
    
        loss_arr.append(running_loss / len(train_dataloader))
    
    print("Training complete!")
    return loss_arr
    

In [71]:
model = ModifiedModel(base_model)

optimizer = AdamW(model.parameters(), lr=5e-3)
loss_metric = nn.MSELoss()

In [72]:
train(model, train_dataloader, loss_metric, optimizer, num_epochs=5)

  0%|          | 0/37365 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [36]:
# print(torch.cuda.memory_summary(device=None, abbreviated=True))

In [18]:
# Empty the GPU cache
torch.cuda.empty_cache()

# Reset the peak memory stats
torch.cuda.reset_peak_memory_stats()

In [ ]:
# Save the model
torch.save(model.state_dict(), './qwen_gsm8k_finetuned/pytorch_model.bin')
tokenizer.save_pretrained("./qwen_gsm8k_finetuned")

In [ ]:
# Load the model again
model = ModifiedModel(base_model)  # Recreate your modified model
model.load_state_dict(torch.load('./qwen_gsm8k_finetuned/pytorch_model.bin'))

# Evaluate

In [25]:
def evaluate(model, dataloader, device):
    model = model.to(device)
    model.eval()

    num_training_steps = len(dataloader)
    progress_bar = tqdm(range(num_training_steps))

    running_loss = 0
    nCorrect = 0

    with torch.no_grad():
        for i, sample in enumerate(dataloader):
            batch = {k: torch.tensor(v).to(device) for k, v in sample.items() if k != 'answer'}
            
            output = model(batch)
            if isinstance(output, tuple):  # Ensure proper indexing
                output = output[0]
            
            # Ensure shape consistency for loss calculation
            loss = loss_metric(output.view(-1, 1).float(), sample['answer'].view(-1, 1).to(device).float())
            nCorrect += (output.view(-1, 1).float() == sample['answer'].view(-1, 1).to(device).float()) 

            optimizer.step()
            optimizer.zero_grad()
            progress_bar.update(1)

            running_loss += loss.item()
            if (i % 100 == 0):
                print(f"Step {i}: Loss = {loss.item()}")

    return nCorrect / len(dataloader)

In [26]:
acc = evaluate(model, eval_dataloader, device)

  0%|          | 0/1319 [00:00<?, ?it/s]

Step 0: Loss = 12708.708984375
Step 100: Loss = 899.7828979492188
Step 200: Loss = 3.757678508758545
Step 300: Loss = 47496.15625
Step 400: Loss = 247942.71875
Step 500: Loss = 288.5423278808594
Step 600: Loss = 104895.5078125
Step 700: Loss = 625088448.0
Step 800: Loss = 128.3656768798828
Step 900: Loss = 2277.062744140625
Step 1000: Loss = 15.132217407226562
Step 1100: Loss = 658.2655029296875
Step 1200: Loss = 487.9842834472656
Step 1300: Loss = 668.4796142578125


In [27]:
print("Test Accuracy:" , acc)

Test Accuracy: tensor([[0.]], device='cuda:0')
